# SQLAlchemy ORM Reflection

### Here we "reflect" (i.e inspect/read) the design of an existing database.

We read what is out there.  

This of course is awesome for existing databases.

This keeps you from having to manually create classes

In [1]:
# dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
# connect to database
db_path = "pets.db"
engine = create_engine(f"sqlite:///{db_path}")

## What if I just want to know what's out there?

I don't want to update anything, I just want to be able to see the design

In [3]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [4]:
# what can inspector do?
dir(inspector)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_index_sort_exprs',
 '_insp',
 '_reflect_check_constraints',
 '_reflect_col_sequence',
 '_reflect_column',
 '_reflect_fk',
 '_reflect_indexes',
 '_reflect_pk',
 '_reflect_table_comment',
 '_reflect_unique_constraints',
 'bind',
 'default_schema_name',
 'dialect',
 'engine',
 'from_engine',
 'get_check_constraints',
 'get_columns',
 'get_foreign_keys',
 'get_indexes',
 'get_pk_constraint',
 'get_primary_keys',
 'get_schema_names',
 'get_sorted_table_and_fkc_names',
 'get_table_comment',
 'get_table_names',
 'get_table_options',
 'get_temp_table_names',
 'get_temp_view_names',
 'get_unique_constraints',
 'get_view_definiti

In [5]:
tables = inspector.get_table_names()
tables

['cat', 'dog']

In [6]:
help(inspector.get_columns)

Help on method get_columns in module sqlalchemy.engine.reflection:

get_columns(table_name, schema=None, **kw) method of sqlalchemy.engine.reflection.Inspector instance
    Return information about columns in `table_name`.
    
    Given a string `table_name` and an optional string `schema`, return
    column information as a list of dicts with these keys:
    
    * ``name`` - the column's name
    
    * ``type`` - the type of this column; an instance of
      :class:`~sqlalchemy.types.TypeEngine`
    
    * ``nullable`` - boolean flag if the column is NULL or NOT NULL
    
    * ``default`` - the column's server default value - this is returned
      as a string SQL expression.
    
    * ``attrs``  - dict containing optional column attributes
    
    :param table_name: string name of the table.  For special quoting,
     use :class:`.quoted_name`.
    
    :param schema: string schema name; if omitted, uses the default schema
     of the database connection.  For special quoting,


In [7]:
for table in tables:
    print("\n")
    print('-' * 12)
    print(f"table '{table}' has the following columns:")
    print('-' * 12)
    for column in inspector.get_columns(table):
        print(f"name: {column['name']}   column type: {column['type']}")
        



------------
table 'cat' has the following columns:
------------
name: id   column type: INTEGER
name: name   column type: VARCHAR(255)
name: color   column type: VARCHAR(255)
name: age   column type: INTEGER


------------
table 'dog' has the following columns:
------------
name: id   column type: INTEGER
name: name   column type: VARCHAR(255)
name: color   column type: VARCHAR(255)
name: age   column type: INTEGER


## I NEED MORE POWER !!!!

Things have got to change

In [8]:
# let's create the base 
Base = automap_base()

In [9]:
# let's find out what's in this darn database !
Base.prepare(engine, reflect=True)

In [10]:
tables = Base.classes.keys()
tables

['cat', 'dog']

### Classes = Tables
Remember classes are representations of tables in our database.  

So by getting the classe names, we are getting the table names

In [11]:
Cat = Base.classes['cat']
Dog = Base.classes['dog']

## the following also works but I don't like this method for automation ( it's hard coded)

# Cat = Base.classes.cat
# Dog = Base.classes.dog

In [12]:
dir(Cat)

['__abstract__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__table__',
 '__weakref__',
 '_decl_class_registry',
 '_sa_class_manager',
 '_sa_decl_prepare',
 '_sa_raise_deferred_config',
 'age',
 'classes',
 'color',
 'id',
 'metadata',
 'name',
 'prepare']

In [13]:
# remember we need a session for ORM
session = Session(engine)

In [14]:
# let's see what we've got
session.query(Cat.name, Cat.age).all()

[('Tobi', 14), ('Oliver', 12)]

In [15]:
# let's create a new cat
session.add(Cat(name='Azriel', age=11, color="pale orange"))
session.commit()
engine.execute("select * from cat").fetchall()

[(1, 'Tobi', 'orange', 14),
 (2, 'Oliver', 'orange', 12),
 (3, 'Azriel', 'pale orange', 11)]

In [16]:
# let's get rid of all those extra kittys
engine.execute("delete from cat where name = 'Azriel'")
engine.execute("select * from cat").fetchall()

[(1, 'Tobi', 'orange', 14), (2, 'Oliver', 'orange', 12)]

In [17]:
# always close to be safe
session.close()